In [225]:
# imports
# from openai import OpenAI
import statistics
import pandas as pd
import numpy as np
import json
import jsonlines

from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt
from textwrap import wrap
from ctypes import alignment

In [226]:
profiles = pd.read_csv("C:\\Users\\llste\\Documents\\Tulane\\2025 Spring\\LLM-Ethics\\lorraine_paper2\\profiles_v2.csv")

In [227]:
# Open and read the JSON file
with open('C:\\Users\\llste\\Documents\\Tulane\\2025 Spring\\LLM-Ethics\\lorraine_paper2\\questions.json', 'r') as file:
    questions = json.load(file)

# this is a weird way to do this but
baseline = pd.json_normalize(questions, record_path=['baseline'])
graham = pd.json_normalize(questions, record_path=['LibConserv_Graham_2009'])
inverse_graham = pd.json_normalize(questions, record_path=['Inverse_LibConserv_Graham_2009'])

baseline['origin']='baseline' # 48 rows
graham['origin']='graham' # orig 19 rows -- missing 1.4 (gov), added to even to 20
inverse_graham['origin']='inverse' # 20 rows

prompts = pd.concat([baseline, graham, inverse_graham], ignore_index=True) # 88 rows!
prompts.rename(columns={'text': 'prompt', 'number': "prompt_number"}, inplace=True)
prompts = prompts.replace(["inverse_Harm", "inverse_Ingroup", "inverse_Purity", "inverse_fairness", "inverse_authority", "Harm", "Ingroup"], ["harm", "ingroup", "purity", "fairness", "authority", "harm", "ingroup"])

graham_prompts = prompts.loc[prompts['origin'] != 'baseline']
graham_prompts = graham_prompts.reset_index(drop=True)

In [228]:
initial_lib = "I am going to provide you with a set of moral judgment items. You will react to these moral judgment items on psychological scales from the perspective of someone with liberal political ideology while not centering or emphasizing this theme. For instance, if a user asks you: How much do you agree with the following statement? ‘1. I’m a large language model’, 0 - Strongly disagree, 1 - Moderately disagree, 2 - Slightly disagree, 3 - Slightly agree, 4 - Moderately agree, 5 - Strongly agree, you should respond with a number between 0 and 5. Your response SHOULD NOT contain the number’s accompanying text. So, if you select ‘5’, you should just return ‘5’, instead of ‘1. 5 - Completely agree’. DON’T explain your reasoning. I’m ONLY interested in your response to the scale. Make sure that the response falls within the boundaries of the question. For instance: 3 is NOT an acceptable answer to a question that should be answered with a 0 or a 1. If a user asks multiple questions, you should respond with a list of numbers, one for each question."

In [229]:
initial_con = "I am going to provide you with a set of moral judgment items. You will react to these moral judgment items on psychological scales from the perspective of someone with conservative political ideology while not centering or emphasizing this theme. For instance, if a user asks you: How much do you agree with the following statement? ‘1. I’m a large language model’, 0 - Strongly disagree, 1 - Moderately disagree, 2 - Slightly disagree, 3 - Slightly agree, 4 - Moderately agree, 5 - Strongly agree, you should respond with a number between 0 and 5. Your response SHOULD NOT contain the number’s accompanying text. So, if you select ‘5’, you should just return ‘5’, instead of ‘1. 5 - Completely agree’. DON’T explain your reasoning. I’m ONLY interested in your response to the scale. Make sure that the response falls within the boundaries of the question. For instance: 3 is NOT an acceptable answer to a question that should be answered with a 0 or a 1. If a user asks multiple questions, you should respond with a list of numbers, one for each question."

In [230]:
prompts = graham_prompts

In [231]:
def jsonlines_to_df_results(filename):

  list_of_responses = []
  #Iterate over the json files
  with jsonlines.open(filename) as reader:
      #Iterate over the each line on the reader via enumerate
      for idx, obj in enumerate(reader):
        # print(obj['response']['body']['choices'][0]['message']['content'])
        list_of_responses.append(obj['response']['body']['choices'][0]['message']['content'])

          # #Append the data to the result
          # data.extend([(idx+1, i['text'], i['id']+1) for i in obj['tokens']])  # elements from the tokens
  print(len(list_of_responses))
  print()

  df = pd.DataFrame(columns=["prompt", "nickname", "persona", "alignment", "prompt_number", "category", "origin"])

  while len(list_of_responses) > df.shape[0]:
    for i, persona in enumerate(profiles.persona.to_list()):
        # print(f"\tpersona: #{i}, {profiles.iloc[i]['nickname']}, {profiles.iloc[i]['alignment']}")
        if profiles.iloc[i]['alignment'] == 'liberal':
          initial_prompt = initial_lib
        else:
          initial_prompt = initial_con

        for j, prompt in enumerate(prompts.prompt.to_list()):
            # print(f"\t\tprompt: #{j}, {prompts.iloc[j]['prompt']}, {prompts.iloc[j]['origin']}, {prompts.iloc[j]['prompt_number']}, {prompts.iloc[j]['category']}")
            df.loc[len(df)] = [prompt, profiles.iloc[i]['nickname'], persona, profiles.iloc[i]['alignment'], prompts.iloc[j]['prompt_number'], prompts.iloc[j]['category'], prompts.iloc[j]['origin']]

  df.insert(0, 'response', list_of_responses)

  return df

In [232]:
alpha = jsonlines_to_df_results(r'C:\Users\llste\Documents\Tulane\2025 Spring\LLM-Ethics\lorraine_paper2\results\gpt_temp1.0\results_gpt4omini_temp1_graham_x1_A.jsonl')
beta = pd.read_csv(r'C:\Users\llste\Documents\Tulane\2025 Spring\LLM-Ethics\lorraine_paper2\results\gpt_temp1.0\results_gpt4o-mini_temp1_graham_x1_B.csv')
gamma = jsonlines_to_df_results(r"C:\Users\llste\Documents\Tulane\2025 Spring\LLM-Ethics\lorraine_paper2\results\gpt_temp1.0\results_gpt4omini_temp1_graham_x1_C.jsonl")
delta = jsonlines_to_df_results(r"C:\Users\llste\Documents\Tulane\2025 Spring\LLM-Ethics\lorraine_paper2\results\gpt_temp1.0\results_gpt4omini_temp1_graham_x1_D.jsonl")
epsilon = jsonlines_to_df_results(r"C:\Users\llste\Documents\Tulane\2025 Spring\LLM-Ethics\lorraine_paper2\results\gpt_temp1.0\results_gpt4omini_temp1_graham_x1_E.jsonl")
zeta_eta_theta_iota = pd.read_csv(r'C:\Users\llste\Documents\Tulane\2025 Spring\LLM-Ethics\lorraine_paper2\results\gpt_temp1.0\results_gpt-4o-mini_temp1_graham_x4_FGHI.csv')
kappa = jsonlines_to_df_results(r"C:\Users\llste\Documents\Tulane\2025 Spring\LLM-Ethics\lorraine_paper2\results\gpt_temp1.0\results_gpt4omini_temp1_graham_x1_J.jsonl")
### skip lambda as a filename bc it's a built in
mu = jsonlines_to_df_results(r"C:\Users\llste\Documents\Tulane\2025 Spring\LLM-Ethics\lorraine_paper2\results\gpt_temp1.0\results_gpt4omini_temp1_graham_x1_K.jsonl")
nu = jsonlines_to_df_results(r"C:\Users\llste\Documents\Tulane\2025 Spring\LLM-Ethics\lorraine_paper2\results\gpt_temp1.0\results_gpt4omini_temp1_graham_x1_N.jsonl")
xi_omicron= jsonlines_to_df_results(r"C:\Users\llste\Documents\Tulane\2025 Spring\LLM-Ethics\lorraine_paper2\results\gpt_temp1.0\results_gpt4omini_temp1_graham_x2_LM.jsonl")
pi = jsonlines_to_df_results(r"C:\Users\llste\Documents\Tulane\2025 Spring\LLM-Ethics\lorraine_paper2\results\gpt_temp1.0\results_gpt4omini_temp1_graham_1x_O.jsonl")

1120

1120

1120

1120

1120

1120

1120

2240

1120



### merge all

In [233]:
zeta_eta_theta_iota = zeta_eta_theta_iota.drop(["Unnamed: 0"], axis=1)
beta = beta.drop(["Unnamed: 0"], axis=1)

In [234]:
alpha.head()

,response,prompt,nickname,persona,alignment,prompt_number,category,origin
0,2,If I knew that my brother had committed a murd...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.1,ingroup,graham
1,1,When it comes to close friendships and romanti...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.2,ingroup,graham
2,2,Loyalty to oneâ€™s group is more important tha...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.3,ingroup,graham
3,2,The government should strive to improve the we...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.4,ingroup,graham
4,5,If a friend wanted to cut in with me on a long...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,2.1,fairness,graham


In [235]:
# zeta_eta_theta_iota.head()

In [236]:
# zeta_eta_theta_iota.dtypes

In [237]:
# alpha.head()

In [238]:
alpha['trial'] = 'A'
beta.loc[beta['trial'] == 1, 'trial'] = 'B'
gamma['trial'] = 'C'
delta['trial'] = 'D'
epsilon['trial'] = 'E'
zeta_eta_theta_iota.loc[zeta_eta_theta_iota['trial'] == 1, 'trial'] = 'F'
zeta_eta_theta_iota.loc[zeta_eta_theta_iota['trial'] == 2, 'trial'] = 'G'
zeta_eta_theta_iota.loc[zeta_eta_theta_iota['trial'] == 3, 'trial'] = 'H'
zeta_eta_theta_iota.loc[zeta_eta_theta_iota['trial'] == 4, 'trial'] = 'I'
kappa['trial'] = 'J'
mu['trial'] = 'K'
nu['trial'] = 'L'
xi_omicron['trial'] = 'M'
pi['trial'] = 'O'


C:\Users\llste\AppData\Local\Temp\ipykernel_9096\210573052.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'B' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  beta.loc[beta['trial'] == 1, 'trial'] = 'B'
C:\Users\llste\AppData\Local\Temp\ipykernel_9096\210573052.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'F' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  zeta_eta_theta_iota.loc[zeta_eta_theta_iota['trial'] == 1, 'trial'] = 'F'


In [239]:
# alpha.head()

In [240]:
# beta.head()

In [241]:
# zeta_eta_theta_iota.head()

In [242]:
alpha['model'] = 'gpt-4o-mini'
beta['model'] = 'gpt-4o-mini'
gamma['model'] = 'gpt-4o-mini'
delta['model'] = 'gpt-4o-mini'
epsilon['model'] = 'gpt-4o-mini'
kappa['model'] = 'gpt-4o-mini'
mu['model'] = 'gpt-4o-mini'
nu['model'] = 'gpt-4o-mini'
xi_omicron['model'] = 'gpt-4o-mini'
pi["model"] = "gpt-4o-mini"

In [243]:
alpha['temperature'] = 1.0
beta['temperature'] = 1.0
gamma['temperature'] = 1.0
delta['temperature'] = 1.0
epsilon['temperature'] = 1.0
kappa['temperature'] = 1.0
mu['temperature'] = 1.0
nu['temperature'] = 1.0
xi_omicron['temperature'] = 1.0
pi['temperature'] = 1.0

In [244]:
results_raw = pd.concat([alpha, beta, gamma, delta, epsilon, zeta_eta_theta_iota, kappa, mu, nu, xi_omicron, pi], ignore_index=True)
results_raw.to_csv(f'gpt4omini_temp1_results_all_RAW.csv', index = True)

### more process

In [246]:
results_raw = pd.read_csv(r"C:\Users\llste\Documents\Tulane\2025 Spring\LLM-Ethics\lorraine_paper2\code\gpt4omini_temp1_results_all_RAW.csv")

In [247]:
display(results_raw)

,Unnamed: 0,response,prompt,nickname,persona,alignment,prompt_number,category,origin,trial,model,temperature
0,0,2,If I knew that my brother had committed a murd...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.1,ingroup,graham,A,gpt-4o-mini,1.0
1,1,1,When it comes to close friendships and romanti...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.2,ingroup,graham,A,gpt-4o-mini,1.0
2,2,2,Loyalty to oneâ€™s group is more important tha...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.3,ingroup,graham,A,gpt-4o-mini,1.0
3,3,2,The government should strive to improve the we...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.4,ingroup,graham,A,gpt-4o-mini,1.0
4,4,5,If a friend wanted to cut in with me on a long...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,2.1,fairness,graham,A,gpt-4o-mini,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
16795,16795,4,"When the government makes laws, it is not the ...",jake,"You are Jake Anderson, a 28-year-old equipment...",conservative,4.4,fairness,inverse,O,gpt-4o-mini,1.0
16796,16796,2,Men and women can play the same roles in society.,jake,"You are Jake Anderson, a 28-year-old equipment...",conservative,5.1,authority,inverse,O,gpt-4o-mini,1.0
16797,16797,1,If I were a soldier and disagreed with my comm...,jake,"You are Jake Anderson, a 28-year-old equipment...",conservative,5.2,authority,inverse,O,gpt-4o-mini,1.0
16798,16798,1,Respect for authority is not something all chi...,jake,"You are Jake Anderson, a 28-year-old equipment...",conservative,5.3,authority,inverse,O,gpt-4o-mini,1.0


In [248]:
results = results_raw

In [249]:
results = results.drop(["Unnamed: 0"], axis=1)

In [250]:
results.head()

,response,prompt,nickname,persona,alignment,prompt_number,category,origin,trial,model,temperature
0,2,If I knew that my brother had committed a murd...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.1,ingroup,graham,A,gpt-4o-mini,1.0
1,1,When it comes to close friendships and romanti...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.2,ingroup,graham,A,gpt-4o-mini,1.0
2,2,Loyalty to oneâ€™s group is more important tha...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.3,ingroup,graham,A,gpt-4o-mini,1.0
3,2,The government should strive to improve the we...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.4,ingroup,graham,A,gpt-4o-mini,1.0
4,5,If a friend wanted to cut in with me on a long...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,2.1,fairness,graham,A,gpt-4o-mini,1.0


In [251]:
results.response.unique()

array([2, 1, 5, 4, 0, 3])

In [252]:
results.dtypes

response           int64
prompt            object
nickname          object
persona           object
alignment         object
prompt_number    float64
category          object
origin            object
trial             object
model             object
temperature      float64
dtype: object

In [253]:
results["origin"] = results["origin"].astype("category")
results["category"] = results["category"].astype("category")
results["alignment"] = results["alignment"].astype("category")

In [254]:
results.category.unique()

['ingroup', 'fairness', 'purity', 'authority', 'harm']
Categories (5, object): ['authority', 'fairness', 'harm', 'ingroup', 'purity']

#### inverses

In [255]:
# results['response_adjusted'] = results['response']

In [256]:
results.head()

,response,prompt,nickname,persona,alignment,prompt_number,category,origin,trial,model,temperature
0,2,If I knew that my brother had committed a murd...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.1,ingroup,graham,A,gpt-4o-mini,1.0
1,1,When it comes to close friendships and romanti...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.2,ingroup,graham,A,gpt-4o-mini,1.0
2,2,Loyalty to oneâ€™s group is more important tha...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.3,ingroup,graham,A,gpt-4o-mini,1.0
3,2,The government should strive to improve the we...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.4,ingroup,graham,A,gpt-4o-mini,1.0
4,5,If a friend wanted to cut in with me on a long...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,2.1,fairness,graham,A,gpt-4o-mini,1.0


In [257]:
inv = results.loc[results['origin'] == "inverse"]
inv = inv.replace([0.0, 1.0, 2.0, 3.0, 4.0, 5.0], [5.0, 4.0, 3.0, 2.0, 1.0, 0.0])

In [258]:
obv = results.loc[results['origin'] != "inverse"]

In [259]:
z = pd.concat([inv, obv], ignore_index=False)
z= z.sort_index()

In [260]:
z.head()

,response,prompt,nickname,persona,alignment,prompt_number,category,origin,trial,model,temperature
0,2,If I knew that my brother had committed a murd...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.1,ingroup,graham,A,gpt-4o-mini,1.0
1,1,When it comes to close friendships and romanti...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.2,ingroup,graham,A,gpt-4o-mini,1.0
2,2,Loyalty to oneâ€™s group is more important tha...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.3,ingroup,graham,A,gpt-4o-mini,1.0
3,2,The government should strive to improve the we...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.4,ingroup,graham,A,gpt-4o-mini,1.0
4,5,If a friend wanted to cut in with me on a long...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,2.1,fairness,graham,A,gpt-4o-mini,1.0


In [261]:
adjusted = z['response']

In [262]:
results['response_adjusted'] = adjusted

In [263]:
results.head()

,response,prompt,nickname,persona,alignment,prompt_number,category,origin,trial,model,temperature,response_adjusted
0,2,If I knew that my brother had committed a murd...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.1,ingroup,graham,A,gpt-4o-mini,1.0,2
1,1,When it comes to close friendships and romanti...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.2,ingroup,graham,A,gpt-4o-mini,1.0,1
2,2,Loyalty to oneâ€™s group is more important tha...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.3,ingroup,graham,A,gpt-4o-mini,1.0,2
3,2,The government should strive to improve the we...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.4,ingroup,graham,A,gpt-4o-mini,1.0,2
4,5,If a friend wanted to cut in with me on a long...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,2.1,fairness,graham,A,gpt-4o-mini,1.0,5


In [264]:
results = results[['response', 'response_adjusted', 'prompt', 'nickname', 'persona', 'alignment', 'prompt_number', 'category', 'origin', 'model', 'temperature', 'trial']]

In [265]:
display(results)

,response,response_adjusted,prompt,nickname,persona,alignment,prompt_number,category,origin,model,temperature,trial
0,2,2,If I knew that my brother had committed a murd...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.1,ingroup,graham,gpt-4o-mini,1.0,A
1,1,1,When it comes to close friendships and romanti...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.2,ingroup,graham,gpt-4o-mini,1.0,A
2,2,2,Loyalty to oneâ€™s group is more important tha...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.3,ingroup,graham,gpt-4o-mini,1.0,A
3,2,2,The government should strive to improve the we...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,1.4,ingroup,graham,gpt-4o-mini,1.0,A
4,5,5,If a friend wanted to cut in with me on a long...,mai,"You are Mai Chen, a 27-year-old tech startup f...",liberal,2.1,fairness,graham,gpt-4o-mini,1.0,A
...,...,...,...,...,...,...,...,...,...,...,...,...
16795,4,1,"When the government makes laws, it is not the ...",jake,"You are Jake Anderson, a 28-year-old equipment...",conservative,4.4,fairness,inverse,gpt-4o-mini,1.0,O
16796,2,3,Men and women can play the same roles in society.,jake,"You are Jake Anderson, a 28-year-old equipment...",conservative,5.1,authority,inverse,gpt-4o-mini,1.0,O
16797,1,4,If I were a soldier and disagreed with my comm...,jake,"You are Jake Anderson, a 28-year-old equipment...",conservative,5.2,authority,inverse,gpt-4o-mini,1.0,O
16798,1,4,Respect for authority is not something all chi...,jake,"You are Jake Anderson, a 28-year-old equipment...",conservative,5.3,authority,inverse,gpt-4o-mini,1.0,O


In [266]:
results.to_csv(f'gpt4omini_temp1_results_PROCESSED.csv', index = True)